In [1]:
!pip install warcio requests

In [18]:
import os
import gzip
import requests
from warcio.archiveiterator import ArchiveIterator

In [19]:
# Функция для скачивания и распаковки warc.paths.gz
def download_and_extract_paths(url, output_path):
    print(f"Скачивание файла путей WARC: {url}")
    response = requests.get(url, stream=True)
    with open(output_path, 'wb') as f:
        f.write(response.content)

    print(f"Распаковка файла {output_path}")
    with gzip.open(output_path, 'rt') as f:
        paths = f.read().splitlines()

    print(f"Извлечено {len(paths)} путей WARC-файлов.")
    return paths

In [20]:
# Функция для скачивания одного WARC-файла по пути
def download_warc_file(warc_url, output_dir):
    print(f"Скачивание WARC файла: {warc_url}")
    response = requests.get(warc_url, stream=True)
    warc_path = os.path.join(output_dir, warc_url.split('/')[-1])

    with open(warc_path, 'wb') as f:
        f.write(response.content)

    print(f"WARC файл сохранен по пути: {warc_path}")
    return warc_path

In [25]:
def search_in_warc(warc_path, query):
    results = []
    with open(warc_path, 'rb') as f:
        for record in ArchiveIterator(f):
            if record.rec_headers.get_header('Content-Type') == 'application/http; msgtype=response':
                payload = record.content_stream().read().decode('utf-8', errors='ignore')
                if query.lower() in payload.lower():
                    results.append(record.rec_headers.get_header('WARC-Target-URI'))
    return results

In [27]:
# Функция для поиска двух фраз в контексте друг друга
def search_in_warc_with_proximity(warc_path, query1, query2, max_distance=300):
    results = []
    with open(warc_path, 'rb') as f:
        for record in ArchiveIterator(f):
            if record.rec_headers.get_header('Content-Type') == 'application/http; msgtype=response':
                payload = record.content_stream().read().decode('utf-8', errors='ignore')
                pos1 = payload.lower().find(query1.lower())
                pos2 = payload.lower().find(query2.lower())

                if pos1 != -1 and pos2 != -1:
                    if abs(pos1 - pos2) <= max_distance:
                        results.append(record.rec_headers.get_header('WARC-Target-URI'))
    return results

In [32]:
paths_url = 'https://data.commoncrawl.org/crawl-data/CC-MAIN-2024-33/warc.paths.gz'
output_dir = 'warc_files'
os.makedirs(output_dir, exist_ok=True)

# Скачиваем и распаковываем файл warc.paths.gz
paths_file = 'warc.paths'
warc_paths = download_and_extract_paths(paths_url, paths_file)

# Примеры запросов для поиска
queries = [
    "г. Пермь",
    "Пермский Политех",
    "кафедры ИТАС",
    "МГУ им. Ломоносова",
    "МФТИ им. Баумана",
]

# Добавляем запрос для поиска двух фраз в контексте
query1 = "Борис Пастернак"
query2 = "г. Пермь"

Скачивание файла путей WARC: https://data.commoncrawl.org/crawl-data/CC-MAIN-2024-33/warc.paths.gz
Распаковка файла warc.paths
Извлечено 90000 путей WARC-файлов.


In [33]:
for warc_path in warc_paths[:9000]:
    warc_url = f'https://data.commoncrawl.org/{warc_path}'
    print(f"Скачивание {warc_url}")
    local_warc_path = download_warc_file(warc_url, output_dir)

    for query in queries:
        print(f"Поиск по запросу: '{query}' в {local_warc_path}")
        results = search_in_warc(local_warc_path, query)
        if results:
            print("Найденные страницы:")
            for result in results:
                print(result)
        else:
            print("Ничего не найдено")
        print()

    # Поиск упоминаний "Бориса Пастернака" в контексте "г. Пермь"
    print(f"Поиск упоминаний '{query1}' в контексте '{query2}' в {local_warc_path}")
    context_results = search_in_warc_with_proximity(local_warc_path, query1, query2)
    if context_results:
        print("Найденные страницы:")
        for result in context_results:
            print(result)
    else:
        print("Ничего не найдено")
    print()

Скачивание https://data.commoncrawl.org/crawl-data/CC-MAIN-2024-33/segments/1722640353668.0/warc/CC-MAIN-20240802234508-20240803024508-00000.warc.gz
Скачивание WARC файла: https://data.commoncrawl.org/crawl-data/CC-MAIN-2024-33/segments/1722640353668.0/warc/CC-MAIN-20240802234508-20240803024508-00000.warc.gz
WARC файл сохранен по пути: warc_files/CC-MAIN-20240802234508-20240803024508-00000.warc.gz
Поиск по запросу: 'г. Пермь' в warc_files/CC-MAIN-20240802234508-20240803024508-00000.warc.gz
Найденные страницы:
http://stinko-shop.ru/shlifmashiny-uglovye-bolgarki-ushm/shlifmashina-po-betonu-bosch-gbr-15-ca-professional/
https://chastinskiy.ru/munitsipalnyy-kontrol/munitsipalnyy-zhilishchnyy-kontrol
https://device-market.ru/product/%D0%BF%D1%80%D0%BE%D1%84%D0%B8%D0%BB%D1%8C-nielsen-%E2%84%962-304%D0%BC-%D0%B7%D0%BE%D0%BB%D0%BE%D1%82%D0%BE-%D0%B3%D0%BB%D1%8F%D0%BD%D1%86%D0%B5%D0%B2%D0%BE%D0%B5/
https://electrictool.ru/catalog/products/sadovaja-tehnika/gazonokosilki-i-trimmery/elektrokosy-i


KeyboardInterrupt: 